# Model Factory

> Scripts to get each Generative Model easily

In [ ]:
#| default_exp model_factory

In [ ]:
#| export
#| hide
from orbit_generation.architectures import get_conv5_vae_components, get_conditional_conv5_legit_tsgm_vae_components, get_conv5_legit_tsgm_vae_components, get_inception_time_vae_components
from orbit_generation.vae import BetaVAE, cBetaVAE

import torch

In [ ]:
#| export
def get_model(params):
    model_name = params['model_name']
    model_kwargs = params.get('model_kwargs', {})

    if 'cvae' in model_name.lower():

        beta = model_kwargs.pop('beta', 1.0)

        if model_name == 'cvae_conv5_legit':
            # Passing model_kwargs directly to the function
            encoder, decoder = get_conditional_conv5_legit_tsgm_vae_components(
                seq_len=params['seq_len'], 
                feat_dim=params['feature_dim'], 
                latent_dim=params['latent_dim'],
                **model_kwargs
            )

        else:
            raise ValueError(f"Unknown VAE model: {model_name}")
        
        # Build the VAE using BetaVAE for all VAE types
        cvae = cBetaVAE(
            encoder=encoder,
            decoder=decoder,
            beta=beta,
            loss_fn=model_kwargs.get('loss_fn', None),
            optimizer_cls=model_kwargs.get('optimizer_cls', torch.optim.Adam),
            lr=model_kwargs.get('lr', params.get('lr'))
        )
        
        return cvae
    
    elif 'vae' in model_name.lower():

        beta = model_kwargs.pop('beta', 1.0)

        if model_name == 'inception_time_vae' or model_name == 'inception_time_wp_vae':
            # Determine whether to use WPInceptionTimeVAEEncoder (without pooling)
            without_pooling = (model_name == 'inception_time_wp_vae')
            
            # Passing model_kwargs directly to the function
            encoder, decoder = get_inception_time_vae_components(
                seq_len=params['seq_len'], 
                feat_dim=params['feature_dim'], 
                latent_dim=params['latent_dim'],
                without_pooling=without_pooling,
                **model_kwargs
            )
        
        elif model_name == 'vae_conv5_legit':
            # Passing model_kwargs directly to the function
            encoder, decoder = get_conv5_legit_tsgm_vae_components(
                seq_len=params['seq_len'], 
                feat_dim=params['feature_dim'], 
                latent_dim=params['latent_dim'],
                **model_kwargs
            )

        elif model_name == 'vae_conv5_1':
            # Passing model_kwargs directly to the function
            encoder, decoder = get_conv5_vae_components(
                seq_len=params['seq_len'], 
                feat_dim=params['feature_dim'], 
                latent_dim=params['latent_dim'],
                **model_kwargs
            )

        else:
            raise ValueError(f"Unknown VAE model: {model_name}")
        
        # Build the VAE using BetaVAE for all VAE types
        vae = BetaVAE(
            encoder=encoder,
            decoder=decoder,
            beta=beta,
            loss_fn=model_kwargs.get('loss_fn', None),
            optimizer_cls=model_kwargs.get('optimizer_cls', torch.optim.Adam),
            lr=model_kwargs.get('lr', params.get('lr'))
        )
        
        return vae
    
    else:
        raise ValueError(f"Model name '{model_name}' is not recognized or not supported yet.")

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()